In [ ]:
from tensorflow.keras.optimizers import Adam, SGD

# Modify the model training function to include hyperparameters as arguments
def train_model(learning_rate=0.001, optimizer='Adam', batch_size=32):
    # Rebuild the model
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg', input_shape=(256, 256, 3))
    base_model.trainable = False

    model = Sequential([
        base_model,
        BatchNormalization(),
        Dropout(0.45),
        Dense(220, activation='relu'),
        Dropout(0.25),
        Dense(60, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Choose the optimizer
    if optimizer == 'Adam':
        chosen_optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        chosen_optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        chosen_optimizer = Adamax(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=chosen_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Fit the model
    history = model.fit(
        train_ds,
        epochs=20,
        validation_data=validation_ds,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )

    return model, history

# Example: Train the model with different hyperparameters
model, history = train_model(learning_rate=0.0005, optimizer='SGD', batch_size=16)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply data augmentation to the training dataset
train_ds = datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='binary'
)


In [ ]:
import numpy as np

def analyze_errors(model, dataset, class_labels):
    misclassified_images = []
    misclassified_labels = []

    for images, labels in dataset:
        predictions = (model.predict(images) > 0.5).astype("int32")
        for i in range(len(predictions)):
            if predictions[i] != labels[i]:
                misclassified_images.append(images[i].numpy())
                misclassified_labels.append((labels[i].numpy(), predictions[i][0]))

    return misclassified_images, misclassified_labels

misclassified_images, misclassified_labels = analyze_errors(model, test_ds, class_labels)

# Visualize misclassified images
def visualize_errors(misclassified_images, misclassified_labels, num_images=5):
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        axs[i].imshow(misclassified_images[i].astype("uint8"))
        true_label, predicted_label = misclassified_labels[i]
        axs[i].set_title(f"True: {class_labels[true_label]}, Pred: {class_labels[predicted_label]}")
        axs[i].axis('off')
    plt.show()

visualize_errors(misclassified_images, misclassified_labels, num_images=5)


In [ ]:
# Convert the model to TensorFlow Lite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to a .tflite file
with open('pneumonia_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, dataset):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        predictions = (model.predict(images) > 0.5).astype("int32")
        y_true.extend(labels.numpy())
        y_pred.extend(predictions)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

evaluate_model(model, test_ds)
